In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy


from model import LR
from data import SyntheticDataset, FairnessDataset
from ei_model_dev import FairBatch
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *

In [2]:
dataset = SyntheticDataset(seed=0)

In [3]:
def append_res(d, acc, ei):
    d['accuracy'].append(acc)
    d['ei_disparity'].append(ei)
        
def get_res(d, id, hp):
    res = {'id': [], 'alpha': [], 'lambda': [], 'delta': [], 'accuracy_mean': [], 'accuracy_std': [], 'ei_disparity_mean': [], 'ei_disparity_std': []}
    res['id'].append(id)
    res['alpha'].append(hp['alpha'])
    res['lambda'].append(hp['lambda'])
    res['delta'].append(hp['delta'])
    res['accuracy_mean'].append(np.mean(d['accuracy']))
    res['accuracy_std'].append(np.std(d['accuracy']))
    res['ei_disparity_mean'].append(np.mean(d['ei_disparity']))
    res['ei_disparity_std'].append(np.std(d['ei_disparity']))
    
    return res

def get_model(models):
    weights = []
    bias = []
    for model in models:
        for module in model.layers:
            if hasattr(module, 'weight'):
                weights.append(module.weight.data)
            if hasattr(module, 'bias'):
                bias.append(module.bias.data)
            
    weights = torch.cat(weights).mean(dim=0)
    bias = torch.cat(bias).mean(dim=0)
    return weights, bias

In [4]:
def lr_fb_model_runner(dataset, hp, seeds):
    tau = 0.5
    train_metrics = {'alpha': [], 'accuracy': [], 'ei_disparity': []}
    val_metrics = deepcopy(train_metrics)
    test_metrics = deepcopy(train_metrics)
    ei_models = []
    
    if hp['optimal_effort']:
        effort = Optimal_Effort(hp['delta'])
    else:
        effort = PGD_Effort(hp['delta'])
    
    for seed in seeds:
        train_tensors, val_tensors, test_tensors = dataset.tensor(fold=seed, z_blind=hp['z_blind'])
        train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
        val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
        test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)
        
        model = LR(num_features=train_dataset.X.shape[1])
        ei_model = FairBatch(model, effort, tau)
        
        ei_model.train(
            train_dataset, 
            sensitive_attrs=dataset.sensitive_attrs,
            lamb=hp['lambda'],
            lr=hp['learning_rate'],
            alpha=hp['alpha']
            )
 
        Y_hat, Y_hat_max = ei_model.predict(train_dataset, hp['alpha'], dataset.sensitive_attrs)
        train_acc, train_ei = model_performance(train_dataset.Y.detach().numpy(), train_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(train_metrics, train_acc, train_ei)
        
        Y_hat, Y_hat_max = ei_model.predict(val_dataset, hp['alpha'], dataset.sensitive_attrs)
        val_acc, val_ei = model_performance(val_dataset.Y.detach().numpy(), val_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(val_metrics, val_acc, val_ei)
        
        Y_hat, Y_hat_max = ei_model.predict(test_dataset, hp['alpha'], dataset.sensitive_attrs)
        test_acc, test_ei = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(test_metrics, test_acc, test_ei)
    
        ei_models.append(ei_model.model)
    
    res_train = get_res(train_metrics, 'train', hp)
    res_val = get_res(val_metrics, 'val', hp)
    res_test = get_res(test_metrics, 'test', hp)
    
    
    return res_train, res_val, res_test, ei_models

In [6]:
def fb_tradeoff(dataset, hyper_params, seeds):
    hp = hyper_params.copy()
    result = pd.DataFrame()
    ei_models = []
    
    for alpha in hyper_params['alpha']:
        for lamb in hyper_params['lambda']:
            for delta in hyper_params['delta']:
                hp['alpha'] = alpha
                hp['lambda'] = lamb
                hp['delta'] = delta
                
                train, val, test, models = lr_fb_model_runner(dataset, hp, seeds)
                result = pd.concat((result, pd.DataFrame(train), pd.DataFrame(val), pd.DataFrame(test)))
                ei_models.extend(models)
                print()
    
    return result, ei_models

In [9]:
hyper_params = {}
hyper_params['learning_rate'] = 0.01
hyper_params['delta'] = [0.5]
hyper_params['alpha'] = [0., 0.1]
hyper_params['lambda'] = [0., 1.]
hyper_params['z_blind'] = False
hyper_params['optimal_effort'] = False
seeds = list(range(1))

results, ei_models = fb_tradeoff(dataset, hyper_params, seeds)
results['loss_mean'] = 1 - results['accuracy_mean']
results['alpha'] = results['alpha'].astype(str)

Training [alpha=0.10; lambda=0.00]: 100%|██████████| 100/100 [01:42<00:00,  1.02s/epochs]


Training [alpha=0.10; lambda=1.00]: 100%|██████████| 100/100 [00:17<00:00,  5.66epochs/s]


In [10]:
results

,id,alpha,lambda,delta,accuracy_mean,accuracy_std,ei_disparity_mean,ei_disparity_std,loss_mean
0,train,"[0.0, 0.1]",0.0,0.5,0.784648,0.000117,0.135887,0.003259,0.215352
0,val,"[0.0, 0.1]",0.0,0.5,0.793750,0.000000,0.111781,0.000608,0.206250
0,test,"[0.0, 0.1]",0.0,0.5,0.783250,0.000000,0.131801,0.002974,0.216750
0,train,"[0.0, 0.1]",1.0,0.5,0.499531,0.119453,0.430556,0.430556,0.500469
0,val,"[0.0, 0.1]",1.0,0.5,0.499844,0.135156,0.500000,0.500000,0.500156
0,test,"[0.0, 0.1]",1.0,0.5,0.499875,0.107375,0.444444,0.444444,0.500125


In [ ]:
train_results = results[results['id'] == 'train']
val_results = results[results['id'] == 'val']
test_results = results[results['id'] == 'test']

In [ ]:
test_results_pareto = pd.DataFrame()
for alpha in test_results['alpha'].unique():
    test_results_alpha = test_results[test_results['alpha'] == alpha]
    mask = pareto_frontier(test_results_alpha['loss_mean'], test_results_alpha['ei_disparity_mean'])
    results_alpha_pareto = test_results_alpha.iloc[mask]
    test_results_pareto = pd.concat((test_results_pareto, results_alpha_pareto.sort_values('ei_disparity_mean')))

In [ ]:
px.line(test_results_pareto, x='ei_disparity_mean', y='loss_mean', color='alpha', hover_data='lambda', markers=True)